# PPD: Programação Paralela e Distribuída

Hélio Crestana Guardia - DC / UFSCar - 2023

**Programa**: multiplicação de matrizes

De que maneira o programa da multiplicação de matrizes poderia ser paralelizado com MPI? 

O programa a seguir aprsenta o esqueleto da uma solução para este problema. Como estamos tratando de computação paralela usando computadores distintos, sem áreas de memória compartilhada para as comunicações, precisamos tratar explicitamente de toda comunicação e sincronização entre os processos executando nos diversos nós.

O probleme começa com a decisão de como será feito o particionamento e de quais dados precisam ser passados a quais nós para as manipulações.

Algumas questões a considerar neste problema:

* Quem lê os dados das matrizes ou gera os valores dos elementos? rank 0?
* Como dividir os cálculos entre os diversos processos? Cada um calcula um grupo de linhas da matriz C?
* Como informar a cada processo quantas (e quais?) linhas (ou conjuntos de elementos) ele irá calcular?
* Supondo a divisão do trabalho com o cálculo de linhas, o que o nó de rank 0 precisa enviar para cada nó? Matriz A inteira e apenas as linhas específicas de B, ou a matriz B é enviada inteira a todos?
* E possível usar MPI_Scatter para enviar a matriz B aos demais processos? Será que se o número de elementos da matriz B não for múltiplo do número de processos, a operação de Scatter poderia gerar erros na distribuição das linhas?
* É possível usar MPI_Gather para receber os dados da matriz C calculados pelos nós? 
* É possível usar MPI_Bcast para enviar a todos? O que seria enviado desta forma?
* O nó de rank 0 deve realizar cálculos ou atua apenas como coordenador?
* Como receber os resultados (linhas de C) dos nós? As mensagens podem chegar fora de ordem?
* É necessária alguma sincronização?

Como o modelo de execução de MPI é comumente SPMD, o mesmo código é executado por todos os processos. Como se vê no programa a seguir, contudo, é possível diferenciar atividades dentro do mesmo código, comumente em função dos *ranks* dos processos, para que processos distintos realizem operações distintas.

Usando o Colab mesmo, é possível editar essa versão do programa e executar aqui.

O exemplo de execução apresentado depois do código realiza a execução e a medição do tempo de execução.

In [ ]:
%%writefile mult.c

/* 
** Universidade Federal de Sao Carlos
** PPD: Programação Paralela e Distribuída
** Hélio Crestana Guardia
*/

/*
** Programa : multiplicacao de matrizes
** Objetivo: paralelizacao com MPI
*/

#include <math.h> 
#include <stdlib.h> 
#include <string.h> 
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include "mpi.h"

float *A, *B, *C;

// Informações a serem enviadas pelo rank 0 aos demais processos
struct info {
  int lin_a;    // número de linhas da matriz A 
  int col_a;    // número de colunas da matriz A (= lin_b)
  int lin_b;    // número de linhas da matriz B (= col_a)
  int col_b;    // número de colunas da matriz B
  int inic;     // número da linha inicial que o processo irá calcular
  int numlin;   // número de linhas que serão calculadas pelo processo
} s_info;

int
main(int argc, char *argv[])
{
	int lin_a,col_a,lin_b,col_b,lin_c,col_c;
	int i,j,k, t;
	int numtasks, rank;
	int result, sum;
	int numlin, resto;

	// Todos os processos iniciam a biblioteca e determinam seus ranks na aplicação
 
	result = MPI_Init(&argc,&argv);
	if (result != MPI_SUCCESS) {
		fprintf(stderr,"Erro iniciando MPI: %d\n",result);
		MPI_Abort(MPI_COMM_WORLD, result);
	}
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
	MPI_Comm_rank(MPI_COMM_WORLD, &rank);


/* 
  Atividades do processo com rank 0:
  ---------------------------------
  - determinar dimensões das matrizes
  - alocar espaço e carregar os dados das matrizes na memória local
  - determinar como será a divisão do trabalho entre os processos
  - enviar a cada processo as informações sobre as matrizes e sobre os cálculos que este irá realizar
  - enviar a cada processo as informações das matrizes pertinentes para os cálculos
  - receber resultados dos cálculos
 
  Atividades dos processos com rank > 0:
  ---------------------------------
  - Aguardar (e receber) informações sobre as matrizes e a divisão do trabalho
	- Alocar espaço em memória para armazenar os dados das matrizes que irão receber
  - Receber os dados pertinentes das matrizes, posicionando-os em memória para os cálculos
  - Realizar os cálculos locais
  - Enviar os valores processados de volta ao processo de rank 0
*/
  
	if(rank==0) {

    /********** Atividades do processo com rank 0 *************************/

    // Nó com rank 0 faz a leitura das dimensõe das matrizes

		setbuf(stdout,NULL); // para forçar a exibição imediata dos textos no terminal

		printf("Linhas A: ");
		scanf("%d",&lin_a);
		printf("Colunas A / Linhas B: ");
		scanf("%d",&col_a);
		lin_b = col_a;
		printf("Colunas B: ");
		scanf("%d",&col_b);
		printf("\n");
		lin_c = lin_a;
		col_c = col_b;
 
  	// Nó rank 0 aloca espaço para as matrizes e as preenche de forma aleatória.
    // Numa aplicação efetiva, provavelmente leria dados de arquivos
 
		// Alocacao dinâmica das matrizes, com linhas em sequência 
		A=(float *)malloc(lin_a*col_a*sizeof(float));
		B=(float *)malloc(lin_b*col_b*sizeof(float));
		C=(float *)malloc(lin_c*col_c*sizeof(float));
 
		// Inicia gerador de números aleatórios. Comentar comando a seguir se quiser
		// gerar sempre os mesmos valores para uniformidade nos cálculos.
		srandom(time(NULL));

		for(i=0; i < lin_a * col_a; i++) 
			A[i]=(float)rand() / (float)RAND_MAX; 

		for(i=0; i < lin_b * col_b; i++) 
			B[i]=(float)rand() / (float)RAND_MAX; 
	
	  // Envio das matrizes para os processos. 
    // O que enviar depende de como os cálculos serão divididos.
    // Aqui, considerando divisão das linhas de C
 
		// preenche informações sobre as matrizes
		s_info.lin_a = lin_a;
		s_info.col_a = col_a;
		s_info.lin_b = lin_b;
		s_info.col_b = col_b;

		// informações sobre a linha inicial e o número de linhas dependem do rank

		// Determina quantas e quais linhas cada processo (rank) vai calcular
		// rank 0 não irá participar dos cálculos...

		numlin = lin_a / (numtasks -1);
		resto = lin_a % (numtasks -1);

		// Determina linha inicial e número de linhas para cada processo rank > 0
    // e lhe envia linhas apropriadas

		for(t=1; t < numtasks; t++) {
      
			// Determina informações sobre linhas a calcular pelo processo
			s_info.numlin = numlin;
			if(t <= resto)
				s_info.numlin += 1;  // resto primeiros processos recebem 1 linha a mais
			s_info.inic = (t-1) * numlin;
			if(resto) {
				if(t<=resto)
					s_info.inic += t-1;    // resto primeiros processos recebem 1 linha a mais
				else
					s_info.inic += resto;   // resto primeiros processos recebem 1 linha a mais
			}
      // Envia informações de controle para demais processos rank > 0

			// MPI_Send( &s_info, sizeof(s_info), MPI_INT, t, 1, MPI_COMM_WORLD);
			MPI_Send( &s_info, sizeof(s_info), MPI_CHAR, t, 1, MPI_COMM_WORLD);

			// Envia linhas de A aos processos rank = t (>0)
			// Enviar 1 linha de cada vez ou todas em sequência?
      // Como linhas estão contíguas na memória, poderia usar MPI_Scatter?
			// Fazer o recebimento correspondende nos demais ranks

			for (i=s_info.inic; i < s_info.inic + s_info.numlin; i++)
				MPI_Send( &A[i*lin_a], col_a, MPI_INT, t, 1, MPI_COMM_WORLD);
    }
  
  	// Nessa estratégia de particionamento, todos precisam da matriz B inteira. 
	 	// Como enviá-la, replicando ou em Bcast? 
	  // Bcast da matriz inteira ou linha por linha?
		// Observar que operação de Bcast, coletiva, deve ser realizada por todos os processos
	  for (i=0; i < lin_b; i++) 
		  MPI_Bcast (&B[i*col_b], col_b, MPI_INT, 0, MPI_COMM_WORLD);

    // Recebe resultados finais. Rank 0 recebe linhas de C
	  // É possível receber as linhas de C fora de ordem?
 
  	for(t=1; t < numtasks; t++) {
        
	  	// Determina informações sobre linhas que foram calculadas por cada processo rank > 0 
		 	// Poderia ter salvo essas infos, já calculadas no envio, num vetor de parâmetros...

	  	s_info.numlin = numlin;
	  	if(t <= resto)
	  		s_info.numlin += 1;  // resto primeiros processos recebem 1 linha a mais
	  	s_info.inic = (t-1) * numlin;
	  	if(resto) {
	  		if(t<=resto)
	  			s_info.inic += t-1;    // resto primeiros processos recebem 1 linha a mais
	  		else
	  			s_info.inic += resto;   // resto primeiros processos recebem 1 linha a mais
	  	}

	  	// Recebe as linhas de C calculadas em cada processo rank > 0
      // Podeira fazer o recebimento fora de ordem?
      // Ideia: usar MPI_ANY_SOURCE... requer recebimento em buffer e cópia para
      // posição efetiva...
	  	for (i=s_info.inic; i < s_info.inic + s_info.numlin; i++)
	  		MPI_Recv( &C[i*col_c], col_c, MPI_INT, t, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }

	} else { 

    /********** Atividades dos processos com rank > 0 *************************/

  	// Recebimento das matrizes e parâmetros para os cálculos
                   
		// Recebem informações sobre as matrizes
		// MPI_Recv (&s_info, sizeof(s_info), MPI_INT, 0, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
		MPI_Recv (&s_info, sizeof(s_info), MPI_CHAR, 0, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

	
		printf("%d recebeu: A[%d,%d], B[%d,%d], inic: %d, numlin: %d\n",
			rank, s_info.lin_a, s_info.col_a, s_info.lin_b, s_info.col_b, s_info.inic, s_info.numlin);
	
		// Alocacam espaços para as matrizes. 
		// Alocam todo o espaço ou apenas para conter as linhas que irão manipular?
		// Matrizes A e C precisam de espaço apenas para as linhas que serão manipuladas pelo processo 
  
		// Alocacao dinâmica das matrizes, com linhas em sequência 
		B = (float *)malloc( s_info.lin_b * s_info.col_b * sizeof(float));
		// A = (float *)malloc( s_info.lin_a * s_info.col_a * sizeof(float));
		A = (float *)malloc( s_info.numlin * s_info.col_a * sizeof(float));
		// C = (float *)malloc( s_info.lin_a * s_info.col_b * sizeof(float));
		C = (float *)malloc( s_info.numlin * s_info.col_b * sizeof(float));
 
		// Recebe linhas de A (s_info.numlin), posicionando-as na matriz alocada
		for (i=0; i < s_info.numlin; i++)     
			MPI_Recv( &A[i*s_info.col_a], s_info.col_a, MPI_INT, 0, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
  
		// Recebe as linhas de B enviadas em Bcast
		for (i=0; i < s_info.lin_b; i++) 
			MPI_Bcast (&B[i*s_info.col_b], s_info.col_b, MPI_INT, 0, MPI_COMM_WORLD);

  	// cálculo da multiplicacao, feito pelos processos de rank > 0
  
		// Cada processo calcula s_info.numlin linhas

		for ( i=0; i < s_info.numlin; i++) 
			for ( j=0; j < s_info.col_b; j++) {
				// C[ i * s_info.col_b +j ] = 0; 
				sum = 0;   
				for ( k=0; k < s_info.col_a; k++) 
					// C[ i * s_info.col_b +j ] += A[ i * s_info.col_a +k ] * B[ k * s_info.col_b +j ];
					sum += A[ i * s_info.col_a +k ] * B[ k * s_info.col_b +j ];
				C[ i * s_info.col_b +j ] = sum;
			}
	    
		// Envia linhas da matriz C calculadas localmente ao process rank 0

		for (i=0; i < s_info.numlin; i++)
			MPI_Send( &C[i*s_info.lin_a], s_info.col_b, MPI_INT, 0, 1, MPI_COMM_WORLD);
	}

/* É possível comparar os resultados calculados paralelamente com os resultados
 * calculados de forma sequencial?
 * Hum... se os dados forem float, a ordem das operações pode gerar resultados
 * diferentes...
 */

  // Se quiser testar os resultados produzidos de forma paralela  
// #define DEBUG
 
#ifdef DEBUG
  if(rank==0) {
   // Cálculo sequencial para comparações
  	float *AB=(float *)malloc(lin_c*col_c*sizeof(float));
  	for(i=0; i < lin_c; i++) 
  		for(j=0; j < col_c; j++) {
  			// AB[i*col_c+j]=0;    
				sum = 0;
  			for(k=0; k < col_a; k++) 
  				// AB[i*col_c+j] = AB[i*col_c+j] + A[i*col_a+k] * B[k*col_b+j];
  				sum += A[i*col_a+k] * B[k*col_b+j];
				AB[i*col_c+j] = sum;
  		}
   // Comparação da matriz calculada sequencialmente com a matriz calculada em paralelo
  	for(i=0;i<lin_c;i++)
  		for(j=0;j<col_c;j++)
  			if(C[i*col_c+j] != AB[i*col_c+j])
  				printf("Erro em %d,%d\n",i,j);
  }
#endif
 
	// Todos os processos
 
  // Libera áreas de memória
  free(A); free(B); free(C);
 
	MPI_Finalize();
 
	return(0);
}


In [ ]:
%%writefile dados
1024
1024
1024

In [ ]:
! if [ ! mult -nt mult.c ]; then mpicc mult.c -o mult -O3 ; fi
# Execução com medição de tempo pelo comando time. Observe o tempo total decorrido (real)
! time mpirun -n 5 ./mult < dados 